In [ ]:
#gameId = "doapp"
#gameId = "sfiii3n"
#gameId = "umk3"
gameId = "tektagt"

In [ ]:
import cv2, sys, os
import random
import numpy as np

sys.path.append(os.path.join(os.path.abspath(''), '../../games'))  
sys.path.append(os.path.join(os.path.abspath(''), '../../utils'))
sys.path.append(os.path.join(os.path.abspath(''), '../../pythonGamePadInterface'))

from diambraMameGym import diambraMame
from diambraGamepad import diambraGamepad 
from policies import gamepadPolicy, RLPolicy # To train AI against another AI or HUM

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/mame/"
diambraKwargs["binary_path"] = "../../customMAME/"
diambraKwargs["frame_ratio"] = 1
diambraKwargs["throttle"] = False
diambraKwargs["sound"] = diambraKwargs["throttle"]

# P1
diambraKwargs["player"] = "P2"
# P2
#diambraKwargs["player"] = "P1P2"

In [ ]:
if gameId != "tektagt":
    diambraKwargs["characters"] = ["Random", "Select"]
else:
    diambraKwargs["characters"] = [["Random", "Random"], ["Random", "Random"]]
diambraKwargs["charOutfits"] = [2, 2]

In [ ]:
# Initialize P2 policy = GamePad
gamePad_policy = gamepadPolicy(diambraGamepad)

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"] = None#gamePad_policy
diambraGymKwargs["continue_game"] = 0.0
diambraGymKwargs["show_final"] = False
diambraGymKwargs["gamePads"] = [None, diambraGymKwargs["P2brain"]]
diambraGymKwargs["actionSpace"] = ["discrete", "discrete"]
diambraGymKwargs["attackButCombinations"] = [True, False]
diambraGymKwargs["actBufLen"]             = 12

In [ ]:
envId = gameId + "_Test"
env = diambraMame(envId, diambraKwargs, **diambraGymKwargs)

In [ ]:
# Printing action spaces
for idx in range(2):
    
    if diambraKwargs["player"] != "P1P2" and idx == 1:
        continue
        
    print("Action space P{} = ".format(idx+1), env.action_spaces[idx])
    print("Action space type P{} = ".format(idx+1), env.action_spaces[idx].dtype)
    if diambraGymKwargs["actionSpace"][idx] == "multiDiscrete":
        print("Action space n = ", env.action_spaces[idx].nvec)
    else:
        print("Action space n = ", env.action_spaces[idx].n)

In [ ]:
def showFrame(observation, waitKey = 1):
    obs = np.array(observation).astype(np.float32)/255

    cv2.imshow("image", obs[:, :, ::-1]) #bgr 2 rgb
    cv2.waitKey(waitKey)

In [ ]:
actionsPrintDict = env.print_actions_dict()

observation = env.reset()

In [ ]:
showFrame(observation, 0)

In [ ]:
while True:

    actions = [None, None]
    for idx in range(2):
        
        actions[idx] = env.action_spaces[idx].sample()

        if diambraGymKwargs["actionSpace"][idx] == "discrete":
            move_action, att_action = env.discreteToMultiDiscreteAction(actions[idx])
        else:
            move_action, att_action = actions[idx][0], actions[idx][1]
        
        if diambraKwargs["player"] != "P1P2" and idx == 1:
            continue
            
        print("(P{}) {} {}".format(idx+1, actionsPrintDict[0][move_action],       
                                          actionsPrintDict[1][att_action])) 
        
    if diambraKwargs["player"] != "P1P2" and diambraGymKwargs["actionSpace"][0] == "discrete":
        actions = actions[0]
    else:
        actions = np.append(actions[0], actions[1])    

    observation, reward, done, info = env.step(actions)

    print("Frames shape:", observation.shape)
    print("Reward:", reward)
    print("Actions Buffer P1 = ", info["actionsBufP1"])
    if diambraKwargs["player"] == "P1P2":
        print("Actions Buffer P2 = ", info["actionsBufP2"])
    print("Rewards:", info["rewards"])
    print("Fighting = ", info["fighting"])

    if diambraKwargs["player"] == "P1P2":
        print("Char P1 = ", env.charNames[env.playingCharacters[0]])
        print("Char P2 = ", env.charNames[env.playingCharacters[1]])
    else:
        print("Char = ", env.charNames[env.playingCharacters[env.playerId]])
    
    if gameId == "tektagt":                                                    
        print("healthP1_1 = ", info["healthP1_1"])                             
        print("healthP1_2 = ", info["healthP1_2"])                             
        print("healthP2_1 = ", info["healthP2_1"])                             
        print("healthP2_2 = ", info["healthP2_2"])                             
        print("activeCharP1 = ", info["activeCharP1"])
        print("activeCharP2 = ", info["activeCharP2"])                                                                     
    else:                                                                       
        print("healthP1 = ", info["healthP1"])                                  
        print("healthP2 = ", info["healthP2"])  
        
    print("PositionP1 = ", info["positionP1"])
    print("PositionP2 = ", info["positionP2"])
    print("WinP1 = ", info["winsP1"])
    print("WinP2 = ", info["winsP2"])
    print("Stage = ", info["stage"])
    print("Round done = ", info["round_done"])
    print("Stage done = ", info["stage_done"])
    print("Game done = ", info["game_done"])
    print("Episode done = ", info["episode_done"])
    print("Done = ", done)
    showFrame(observation, 0)
    
    if done:
        print("Resetting Env")
        observation = env.reset()
        showFrame(observation, 0)
        
env.close()